In [ ]:
#@markdown ### RUN GPU

! nvidia-smi
! nvcc -V
! free -h

In [ ]:
#@markdown ### install ollama in google colab (not google drive)


!apt-get update
!apt-get install -y curl wget


!curl -fsSL https://ollama.com/install.sh | sh


!ollama --version


!mkdir -p /tmp/ollama


In [ ]:
#@markdown ### run ollama

# Set environment variables for Ollama to store models in temporary memory
import os
import subprocess
import time

os.environ["OLLAMA_MODELS"] = "/tmp/ollama"

# Launch Ollama service in the background using subprocess
with open('/tmp/ollama.log', 'w') as log_file:
    process = subprocess.Popen(
        ["ollama", "serve"],
        stdout=log_file,
        stderr=log_file,
        preexec_fn=os.setsid  # For execution in a new process group
    )

# Wait a bit for the service to start
time.sleep(5)

# Check service status
try:
    # Try to run a simple ollama command to confirm that the service is running
    check_process = subprocess.run(
        ["ollama", "list"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        timeout=10
    )
    if check_process.returncode == 0:
        print("✅ Ollama service has been successfully launched!")
        print("\nLatest log lines:")
        !tail -n 5 /tmp/ollama.log
    else:
        print("❌ There is a problem launching Ollama:")
        print(check_process.stderr)
except subprocess.TimeoutExpired:
    print("❌ Response time has expired. The service may still be starting.")
    print("\nLatest log lines:")
    !tail -n 5 /tmp/ollama.log

print("\n🔍 Ollama process ID:")
!pgrep -f "ollama serve"

# Display the address and port for accessing the service
print("\n🌐 Ollama service is available at the following address:")
print("http://127.0.0.1:11434")


In [ ]:
#@markdown ### Download ollama models

#@markdown ##### 8B parameters (~4-5GB disk space)
deepseek_r1_8b = False #@param {type:"boolean"}
llama3_8b = False #@param {type:"boolean"}
qwen3_8b = False #@param {type:"boolean"}

#@markdown ##### 8B parameters, 128K context (~4-5GB disk space)
llama3_1_8b = False #@param {type:"boolean"}

#@markdown ##### 7B parameters (~4GB disk space)
mistral_7b = False #@param {type:"boolean"}
gemma_7b = False #@param {type:"boolean"}
falcon_7b = False #@param {type:"boolean"}
codegemma_7b = False #@param {type:"boolean"}

#@markdown ##### 14B parameters (~7-8GB disk space)
deepseek_r1_14b = False #@param {type:"boolean"}
qwen3_14b = False #@param {type:"boolean"}

#@markdown ##### 12B parameters, 128K context (~7GB disk space)
gemma3_12b = False #@param {type:"boolean"}

#@markdown ##### 32B parameters (~16-18GB disk space)
deepseek_r1_32b = False #@param {type:"boolean"}

#@markdown ##### 27B parameters, 128K context (~14-15GB disk space)
gemma3_27b = False #@param {type:"boolean"}

#@markdown ##### 30.5B parameters (~16GB disk space)
qwen3_30b = False #@param {type:"boolean"}

#@markdown ##### 3.8B parameters (~2GB disk space)
phi3_mini = True #@param {type:"boolean"}

#@markdown ### Add custom models
#@markdown Enter custom model names in the format `model1:tag1,model2:tag2`. Example: `vicuna:latest,orca-mini:3b`
custom_models = "" #@param {type:"string"}

#@markdown ### Test prompt settings
#@markdown Enter a test prompt to evaluate models after download
run_test_prompt = False #@param {type:"boolean"}
test_prompt = "surprise me" #@param {type:"string"}

# Dictionary of predefined models and their selection status
models = {
    "deepseek:r1-8b": deepseek_r1_8b,
    "llama3:8b": llama3_8b,
    "qwen3:8b": qwen3_8b,
    "llama3.1:8b": llama3_1_8b,
    "mistral:7b": mistral_7b,
    "gemma:7b": gemma_7b,
    "falcon:7b": falcon_7b,
    "codegemma:7b": codegemma_7b,
    "deepseek-r1:14b": deepseek_r1_14b,
    "qwen3:14b": qwen3_14b,
    "gemma3:12b": gemma3_12b,
    "deepseek-r1:32b": deepseek_r1_32b,
    "gemma3:27b": gemma3_27b,
    "qwen3:30b": qwen3_30b,
    "phi3:mini": phi3_mini
}

# Model size information for display
model_sizes = {
    "deepseek:r1-8b": "~8B parameters (~4-5GB disk space)",
    "llama3:8b": "~8B parameters (~4-5GB disk space)",
    "qwen3:8b": "~8.19B parameters (~5GB disk space)",
    "llama3.1:8b": "~8B parameters, 128K context (~4-5GB disk space)",
    "mistral:7b": "~7B parameters (~4GB disk space)",
    "gemma:7b": "~7B parameters (~4GB disk space)",
    "falcon:7b": "~7B parameters (~4GB disk space)",
    "codegemma:7b": "~7B parameters (~4GB disk space)",
    "deepseek-r1:14b": "~14B parameters (~7-8GB disk space)",
    "qwen3:14b": "~14.8B parameters (~8GB disk space)",
    "gemma3:12b": "~12B parameters, 128K context (~7GB disk space)",
    "deepseek-r1:32b": "~32B parameters (~16-18GB disk space)",
    "gemma3:27b": "~27B parameters, 128K context (~14-15GB disk space)",
    "qwen3:30b": "~30.5B parameters (~16GB disk space)",
    "phi3:mini": "~3.8B parameters (~2GB disk space)"
}

# Keep track of successfully downloaded models
downloaded_model_list = []

# Add custom models to the dictionary
if custom_models.strip():
    # Split input text by commas
    custom_model_list = [model.strip() for model in custom_models.split(',') if model.strip()]

    # Add each custom model to the dictionary with selection status True
    for model_name in custom_model_list:
        models[model_name] = True
        print(f"Custom model '{model_name}' added to download list.")

# Display model information and download selected models
downloaded_models = 0
print("\n### MODEL INFORMATION ###")
print("-" * 60)
print(f"{'MODEL NAME':<20} {'SELECTION':<15} {'SIZE'}")
print("-" * 60)

for model_name, selected in models.items():
    size_info = model_sizes.get(model_name, "Size information not available")
    status = "SELECTED" if selected else "Not selected"
    print(f"{model_name:<20} {status:<15} {size_info}")

print("\n### DOWNLOADING SELECTED MODELS ###")
for model_name, selected in models.items():
    if selected:
        print(f"\nDownloading model {model_name}...")
        !ollama pull {model_name}
        download_status = !echo $?

        if int(download_status[0]) == 0:
            print(f"Model {model_name} downloaded successfully.")
            downloaded_models += 1
            downloaded_model_list.append(model_name)
        else:
            print(f"Error downloading model {model_name}.")

if downloaded_models == 0:
    print("\n⚠️ No models selected for download!")
else:
    print(f"\n✅ {downloaded_models} models downloaded successfully.")

print("\n### List of downloaded models ###")
!ollama list

# Run test prompt if enabled and models were downloaded
if run_test_prompt and downloaded_model_list:
    import time
    import json
    import subprocess
    import re
    from datetime import datetime
    from IPython.display import display, HTML, Markdown

    print("\n\n")
    print("#" * 80)
    print(f"### RUNNING TEST PROMPT: '{test_prompt}' ###")
    print("#" * 80)

    results = {}

    def clean_ansi_codes(text):
        """Remove ANSI escape codes from text"""
        ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
        return ansi_escape.sub('', text)

    for model_name in downloaded_model_list:
        print(f"\n\n## Testing model: {model_name} ##\n")

        # Record start time
        start_time = time.time()

        # Run the model with direct subprocess call for better control
        print("Generating response...")
        try:
            # Run without --format json to avoid the formatting issues
            process = subprocess.Popen(
                ["ollama", "run", model_name, test_prompt],
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True
            )
            model_response, stderr = process.communicate()

            # Clean response
            model_response = clean_ansi_codes(model_response).strip()

        except Exception as e:
            model_response = f"Error running model: {str(e)}"

        # Record end time and calculate duration
        end_time = time.time()
        duration = end_time - start_time

        # Store results
        results[model_name] = {
            "response": model_response,
            "time": duration
        }

        # Display the response
        print(f"\nResponse from {model_name} (took {duration:.2f} seconds):")
        print("-" * 60)
        display(Markdown(model_response))
        print("-" * 60)

    # Comparison summary
    print("\n\n")
    print("#" * 80)
    print("### MODEL COMPARISON SUMMARY ###")
    print("#" * 80)

    print(f"\n{'MODEL':<20} {'RESPONSE TIME':<15} {'RESPONSE LENGTH'}")
    print("-" * 60)

    for model, data in results.items():
        response_length = len(data["response"])
        print(f"{model:<20} {data['time']:.2f}s{' ':<10} {response_length} chars")

    # Save results to file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    result_file = f"model_comparison_{timestamp}.txt"

    with open(result_file, "w") as f:
        f.write(f"TEST PROMPT: {test_prompt}\n")
        f.write(f"DATE: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

        for model, data in results.items():
            f.write(f"MODEL: {model}\n")
            f.write(f"TIME: {data['time']:.2f}s\n")
            f.write(f"RESPONSE LENGTH: {len(data['response'])} chars\n")
            f.write("-" * 60 + "\n")
            f.write(data["response"])
            f.write("\n\n" + "=" * 80 + "\n\n")

    print(f"\nDetailed comparison saved to '{result_file}'")
elif run_test_prompt and not downloaded_model_list:
    print("\n⚠️ No models were successfully downloaded to test.")


In [ ]:
#@markdown ### Install N*N

# Input fields for parameter configuration

# Enter your ngrok authentication token
NGROK_AUTH_TOKEN = "" #@param {type:"string"}

# Storage folder name in Google Drive
gdrive_path = "n8n_data" #@param {type:"string"}
# Authentication settings
N8N_BASIC_AUTH = False
N8N_USERNAME = "admin"
N8N_PASSWORD = "password"


# Install required packages
!pip install pyngrok
from pyngrok import ngrok, conf
import time
import subprocess
import sys

# Connect to Google Drive for persistent storage
from google.colab import drive
print("Connecting to Google Drive...")
drive.mount('/content/drive')

# Set up storage path in Google Drive for n8n data
import os

# Define full Google Drive path
gdrive_full_path = f"/content/drive/MyDrive/{gdrive_path}"

# Create folders if they don't exist
if not os.path.exists(gdrive_full_path):
    os.makedirs(gdrive_full_path)
    print(f"Folder '{gdrive_path}' created in Google Drive.")
else:
    print(f"Folder '{gdrive_path}' already exists in Google Drive.")

# Create necessary subfolders
os.makedirs(f"{gdrive_full_path}/data", exist_ok=True)
os.makedirs(f"{gdrive_full_path}/database", exist_ok=True)
print("Storage folders prepared.")

# Set up ngrok authentication
# Validate ngrok token
if not NGROK_AUTH_TOKEN:
    print("⚠️ ngrok authentication token not entered! Please get your token from https://dashboard.ngrok.com/get-started/your-authtoken.")
    raise ValueError("ngrok authentication token is required.")

conf.get_default().auth_token = NGROK_AUTH_TOKEN
print("ngrok configured successfully.")

# ===================================================================
# Check Node.js version and install if needed
# ===================================================================
def get_node_version():
    try:
        result = subprocess.run(['node', '--version'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if result.returncode == 0:
            version = result.stdout.decode('utf-8').strip()
            return version
        return None
    except:
        return None

def is_n8n_installed():
    try:
        result = subprocess.run(['which', 'n8n'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return result.returncode == 0
    except:
        return False

# Check Node.js version
node_version = get_node_version()
if node_version:
    print(f"Node.js installed: {node_version}")
    # Check if current version is new enough
    if not node_version.startswith('v18.') and not node_version.startswith('v19.') and not node_version.startswith('v20.'):
        print("Installed Node.js version is not sufficient, installing version 18...")
        !curl -fsSL https://deb.nodesource.com/setup_18.x | sudo -E bash -
        !sudo apt-get install -y nodejs
        print("Node.js updated:")
        !node --version
else:
    print("Node.js not installed, installing...")
    !curl -fsSL https://deb.nodesource.com/setup_18.x | sudo -E bash -
    !sudo apt-get install -y nodejs
    print("Node.js installed:")
    !node --version

# Check if n8n is already installed
if is_n8n_installed():
    print("n8n is already installed, no need to reinstall.")
    !n8n --version
else:
    print("n8n not installed, installing...")
    !npm install n8n -g
    print("n8n installed.")
# ===================================================================

# Create n8n configuration file
n8n_config = f"""
module.exports = {{
  database: {{
    type: 'sqlite',
    logging: false,
    sqliteOptions: {{
      path: '{gdrive_full_path}/database/database.sqlite',
    }},
  }},
  executions: {{
    saveDataOnError: 'all',
    saveDataOnSuccess: 'all',
    saveExecutionProgress: true,
    saveManualExecutions: true,
  }},
"""

# Add authentication configuration if enabled
if N8N_BASIC_AUTH:
    n8n_config += f"""
  basic: {{
    auth: {{
      active: true,
      user: '{N8N_USERNAME}',
      password: '{N8N_PASSWORD}',
    }},
  }},
"""

n8n_config += """
  userManagement: {
    isInstanceOwnerSetUp: true,
  },
  diagnostics: {
    enabled: false,
  },
};
"""

# Write n8n configuration file
config_dir = f"{gdrive_full_path}/config"
os.makedirs(config_dir, exist_ok=True)
with open(f"{config_dir}/config.js", 'w') as f:
    f.write(n8n_config)

# Create symbolic link to n8n configuration in home folder
!mkdir -p ~/.n8n
!ln -sf {config_dir}/config.js ~/.n8n/config.js

# Start ngrok to get public URL before starting n8n
print("Starting ngrok tunnel...")
ngrok.kill()  # Kill existing tunnels
tunnel = ngrok.connect(5678, 'http')
public_url = tunnel.public_url
print(f"Tunnel address: {public_url}")

# Set environment variables for n8n
os.environ["N8N_HOST"] = "localhost"
os.environ["N8N_PORT"] = "5678"
os.environ["N8N_PROTOCOL"] = "http"
os.environ["N8N_PATH"] = "/"
os.environ["NODE_ENV"] = "production"
os.environ["N8N_ENCRYPTION_KEY"] = "your-secret-encryption-key"
os.environ["N8N_USER_FOLDER"] = f"{gdrive_full_path}/data"
os.environ["WEBHOOK_URL"] = public_url
# Add this line to use the ngrok URL directly
os.environ["N8N_WEBHOOK_URL"] = public_url

# Launch n8n in the background - REMOVED --tunnel flag
print("Starting n8n...")
!nohup n8n start > {gdrive_full_path}/n8n.log 2>&1 &
print("Waiting for n8n to start...")
time.sleep(30)

# Check if n8n is running
!ps aux | grep n8n

# Display success message and access information
import IPython.display as display

print("\n" + "-"*50)
print("✅ n8n launched successfully!")
print("-"*50)
print(f"🌐 Access n8n via: {public_url}")

if N8N_BASIC_AUTH:
    print(f"👤 Username: {N8N_USERNAME}")
    print(f"🔑 Password: {N8N_PASSWORD}")

print("\n" + "-"*50)
print("🚀 Usage instructions:")
print("-"*50)
print("1️⃣ To use Ollama models in n8n, use an HTTP Request node.")
print("2️⃣ Ollama API address: http://localhost:11434/api/generate")
print("3️⃣ To test Ollama connection in Colab, you can run these commands:")
print("   !ollama list")
print("   !curl -X POST http://localhost:11434/api/generate -d '{\"model\":\"[model-name]\",\"prompt\":\"hello\"}'")
print("\n4️⃣ To maintain access to n8n, keep this notebook open.")
print("5️⃣ For long-term usage, use strategies to prevent Colab from disconnecting.")

# Display n8n interface in an iframe
display.display(display.IFrame(src=public_url, width="100%", height=800))

# Function to view logs
def view_logs():
    print("n8n logs:")
    !tail -n 20 {gdrive_full_path}/n8n.log
    print("\nOllama logs:")
    !tail -n 20 {gdrive_full_path}/ollama.log

# Keep notebook running
print("\nNotebook is maintaining n8n and Ollama. Do not close this tab.")
